In [ ]:
import os
import SimpleITK as sitk
import numpy as np

In [ ]:
def volume_bounding_box(data, gt, expend=20):
    print(data.shape)
    mask = (gt != 0)
    brain_voxels = np.where(mask != 0)
    z, x, y = data.shape

    minZidx = int(np.min(brain_voxels[0]))
    maxZidx = int(np.max(brain_voxels[0]))
    minXidx = int(np.min(brain_voxels[1]))
    maxXidx = int(np.max(brain_voxels[1]))
    minYidx = int(np.min(brain_voxels[2]))
    maxYidx = int(np.max(brain_voxels[2]))

    minZidx_jitterd = max(minZidx - expend, 0)
    maxZidx_jitterd = min(maxZidx + expend, z) + 1
    minXidx_jitterd = max(minXidx - expend, 0)
    maxXidx_jitterd = min(maxXidx + expend, x) + 1
    minYidx_jitterd = max(minYidx - expend, 0)
    maxYidx_jitterd = min(maxYidx + expend, y) + 1

    data_bboxed = data[minZidx_jitterd:maxZidx_jitterd,
                       minXidx_jitterd:maxXidx_jitterd,
                       minYidx_jitterd:maxYidx_jitterd]

    gt_bboxed = gt[minZidx_jitterd:maxZidx_jitterd,
                   minXidx_jitterd:maxXidx_jitterd,
                   minYidx_jitterd:maxYidx_jitterd]

    print([minZidx, maxZidx, minXidx, maxXidx, minYidx, maxYidx])
    print([
        minZidx_jitterd, maxZidx_jitterd, minXidx_jitterd, maxXidx_jitterd,
        minYidx_jitterd, maxYidx_jitterd
    ])

    return data_bboxed, gt_bboxed


if __name__ == '__main__':
    # 动态nii文件根据label获取roi

    # root_dir = r'C:\Users\chenjy4\Desktop\dongtai\videodatanii\label_smooth_by_polarozation1.1'
    # root_dir = r'C:\Users\chenjy4\Desktop\duibi\nii_pred'
    root_dir = r'C:\Users\chenjy4\Desktop\niipred'

    files = os.listdir(os.path.join(root_dir, 'image'))

    for file in files:
        image = sitk.GetArrayFromImage(
            sitk.ReadImage(os.path.join(root_dir, 'image', file)))
        label = sitk.GetArrayFromImage(
            sitk.ReadImage(os.path.join(root_dir, 'label', file)))

        image_roi, label_roi = volume_bounding_box(image, label, 0)

        if not os.path.exists(os.path.join(root_dir, 'image_roi')):
            os.makedirs(os.path.join(root_dir, 'image_roi'))
        if not os.path.exists(os.path.join(root_dir, 'label_roi')):
            os.makedirs(os.path.join(root_dir, 'label_roi'))

        sitk.WriteImage(sitk.GetImageFromArray(image_roi),
                        os.path.join(root_dir, 'image_roi', file))
        sitk.WriteImage(sitk.GetImageFromArray(label_roi),
                        os.path.join(root_dir, 'label_roi', file))
